In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import re
from google.auth.transport.requests import Request
from google.auth.credentials import Credentials
from googleapiclient.discovery import build
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def fetch_prices(url):
    # Initialize a Chrome WebDriver service
    chrome_service = ChromeService('/Users/hesswillcole/code/chromedriver-mac-arm64/chromedriver')
    chrome_service.start()

    # Initialize a Chrome WebDriver with the service
    driver = webdriver.Chrome(service=chrome_service)

    # Load the URL
    driver.get(url)

    # Wait for the page to load
    time.sleep(2)  # Adjust the delay as needed
        
    # Find the desired date element and click on it
    desired_date_text = "May"  # Replace this with the text of the desired date
    desired_date_element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'//td[@class="monthpipe"][contains(., "{desired_date_text}")]')))
    desired_date_element.click()

    dropdown_img = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//img[@class="dropdown-img"]')))
    dropdown_img.click()

    # Wait for the dropdown options to appear
    dropdown_options = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, '//div[@id="number_of_strikes-dplist"]/ul/li')))

    # Iterate through the dropdown options and click on the desired option
    for option in dropdown_options:
        if option.text == '11':
            option.click()
            break

    # Extract HTML content
    page_source = driver.page_source

    # Close the WebDriver
    driver.quit()

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find script tags containing limit price information
    script_tags = soup.find_all('script', type='text/javascript')

    # Define regular expression to extract limit prices
    pattern_limitprice = re.compile(r'limitprice=([0-9]+\.[0-9]+)')

    # Initialize empty lists to store prices
    limit_prices_inorder = []  # Store limit prices in the order they appear
    limit_prices = set()  # Store unique limit prices
    strike_prices = []

    # Find all strike price elements
    strike_price_tds = soup.find_all('td', class_='strikePrice_bg')
    for td in strike_price_tds:
        a_tag = td.find('a')
        if a_tag:
            strike_prices.append(float(a_tag.text.strip()))

    # Extract all limit prices in order of appearance
    limit_prices_script_tags = soup.find_all('script', type='text/javascript')
    for script in limit_prices_script_tags:
        script_content = script.string
        if script_content and '\'Buy' in script_content:  # Only consider 'Buy' tags
            matches_limitprice = pattern_limitprice.findall(script_content)
            for match in matches_limitprice:
                limit_price = float(match)
                limit_prices_inorder.append(limit_price)

    # Remove every even indexed limit price
    unique_limit_prices_inorder = []
    for i in range(len(limit_prices_inorder)):
        if i % 2 != 0:  # Check if the index is odd
            unique_limit_prices_inorder.append(limit_prices_inorder[i])

    return unique_limit_prices_inorder, strike_prices


def main():
    # Set up Google Sheets API authentication (replace with your credentials)
    credentials_file = '/Users/hesswillcole/code/python-to-sheets-419917-9753d7e4ecb0.json'
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(creds)

    # Open the Google Sheet named 'Etrade' and access the 'Hub' sheet to get the stock ticker
    hub_sheet = client.open('Etrade').worksheet('Hub')

    # Assuming the stock symbol is in the first column of the row referenced by Apps Script (starting from row 3)
    starting_row = 3  # Adjust this based on your Apps Script logic

    # Loop through the rows starting from the specified row
    for row_offset in range(0, 4895 - starting_row + 1):  # Adjust the range based on your Apps Script logic
        # Get the stock symbol from the current row
        stock_row = starting_row + row_offset
        stock_symbol = hub_sheet.cell(stock_row, 1).value

        # Construct the URL with the stock ticker
        url = f'https://us.etrade.com/e/t/invest/quotesandresearch?content=3&sym={stock_symbol}'
        
        # Fetch prices
        limit_prices_inorder, strike_prices = fetch_prices(url)

        # Open the sheet corresponding to the stock symbol
        stock_sheet = client.open('Etrade').worksheet(stock_symbol)

        # Clear columns O, P, and Q and the first 15 rows before printing new data
        stock_sheet.update('O1:Q15', [[''] * 3] * 15)

        # Prepare data for the stock sheet
        data = [["Calls", "Strike", "Puts"]]

        for i in range(len(strike_prices)):
            # Ensure limit_prices_inorder and strike_prices have enough elements
            if (2 * i + 1) < len(limit_prices_inorder) and i < len(strike_prices):
                data.append([limit_prices_inorder[2 * i], strike_prices[i], limit_prices_inorder[2 * i + 1]])

        # Output the formatted data to the stock sheet starting at O1
        for row_index, row_data in enumerate(data):
            for col_index, cell_data in enumerate(row_data):
                stock_sheet.update_cell(row_index + 1, col_index + 15, cell_data)  # Adjust column index to start at O (15)

        # Sleep if needed to avoid being blocked by the website
        time.sleep(5)

if __name__ == "__main__":
    main()


/var/folders/8z/jwv0rgr14s309bl8zzhxfsq80000gn/T/ipykernel_33732/3663598367.py:121: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  stock_sheet.update('O1:Q15', [[''] * 3] * 15)


KeyboardInterrupt: 